In [71]:
import pandas as pd

In [72]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)




In [73]:
with open('input.txt','r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [74]:
print(lines[155])

It's me!	Ich bin's.


In [75]:
len(lines)

169813

In [76]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [77]:
num_samples = 10000
num_samples

10000

In [78]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [79]:
input_texts[155],target_texts[155]

("It's me!", "\tIch bin's.\n")

In [81]:
target_texts

['\tHallo!\n',
 '\tGrÃ¼ÃŸ Gott!\n',
 '\tLauf!\n',
 '\tFeuer!\n',
 '\tHilfe!\n',
 '\tZu HÃ¼lf!\n',
 '\tStopp!\n',
 '\tWarte!\n',
 '\tMach weiter.\n',
 '\tHallo!\n',
 '\tIch rannte.\n',
 '\tIch verstehe.\n',
 '\tAha.\n',
 '\tIch probiere es.\n',
 '\tIch hab gewonnen!\n',
 '\tIch habe gewonnen!\n',
 '\tLÃ¤cheln!\n',
 '\tZum Wohl!\n',
 '\tKeine Bewegung!\n',
 '\tStehenbleiben!\n',
 '\tVerstanden?\n',
 '\tEinverstanden?\n',
 '\tEr rannte.\n',
 '\tEr lief.\n',
 '\tMach mit!\n',
 '\tDrÃ¼ck mich!\n',
 '\tNimm mich in den Arm!\n',
 '\tUmarme mich!\n',
 '\tIch fiel.\n',
 '\tIch fiel hin.\n',
 '\tIch stÃ¼rzte.\n',
 '\tIch bin hingefallen.\n',
 '\tIch bin gestÃ¼rzt.\n',
 '\tIch weiÃŸ.\n',
 '\tIch habe gelogen.\n',
 '\tIch habe verloren.\n',
 '\tIch bin 19 Jahre alt.\n',
 '\tIch bin 19.\n',
 "\tMir geht's gut.\n",
 '\tEs geht mir gut.\n',
 '\tIch bin wach.\n',
 '\tIch bin auf.\n',
 '\tUnmÃ¶glich!\n',
 '\tDas gibtâ€™s doch nicht!\n',
 '\tAusgeschlossen!\n',
 '\tIn keinster Weise!\n',
 '\tWirklich?\n

In [82]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 53


In [83]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [84]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [85]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
          # decoder_target_data will be ahead by one timestep
          # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [86]:
import keras,tensorflow
from keras.models import Model
from keras.layers import Input,LSTM,Dense


In [87]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [88]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [89]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [90]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [91]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, None, 90)     0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 256), (None, 334848      input_9[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 256),  355328      input_10[0][0]                   
                                                                 lstm_5[0][1]               

In [92]:
 model.fit([encoder_input_data, decoder_input_data],
           decoder_target_data, batch_size=batch_size,
           epochs=epochs, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 7s 851us/step - loss: 1.0481 - val_loss: 1.0689
Epoch 2/100
8000/8000 [==============================] - 6s 745us/step - loss: 0.8155 - val_loss: 0.8781
Epoch 3/100
8000/8000 [==============================] - 6s 756us/step - loss: 0.6790 - val_loss: 0.7628
Epoch 4/100
8000/8000 [==============================] - 6s 799us/step - loss: 0.6101 - val_loss: 0.7110
Epoch 5/100
8000/8000 [==============================] - 6s 755us/step - loss: 0.5667 - val_loss: 0.6793
Epoch 6/100
8000/8000 [==============================] - 6s 771us/step - loss: 0.5346 - val_loss: 0.6568
Epoch 7/100
8000/8000 [==============================] - 6s 762us/step - loss: 0.5067 - val_loss: 0.6303
Epoch 8/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.4840 - val_loss: 0.6073
Epoch 9/100
8000/8000 [==============================] - 8s 948us/step - loss: 0.4641 - val_loss: 0.5942
Epoch 10/

Epoch 78/100
8000/8000 [==============================] - 6s 796us/step - loss: 0.0902 - val_loss: 0.7567 ETA: 0s - loss: 0.089 - ETA: 0s - loss: 0.0 - ETA: 0s - loss: 
Epoch 79/100
8000/8000 [==============================] - 6s 797us/step - loss: 0.0889 - val_loss: 0.7703
Epoch 80/100
8000/8000 [==============================] - 6s 793us/step - loss: 0.0876 - val_loss: 0.7616
Epoch 81/100
8000/8000 [==============================] - 6s 784us/step - loss: 0.0863 - val_loss: 0.7702
Epoch 82/100
8000/8000 [==============================] - 6s 795us/step - loss: 0.0849 - val_loss: 0.7650
Epoch 83/100
8000/8000 [==============================] - 6s 799us/step - loss: 0.0835 - val_loss: 0.7802
Epoch 84/100
8000/8000 [==============================] - 6s 798us/step - loss: 0.0823 - val_loss: 0.7746
Epoch 85/100
8000/8000 [==============================] - 6s 796us/step - loss: 0.0811 - val_loss: 0.7811
Epoch 86/100
8000/8000 [==============================] - 6s 793us/step - loss: 0.0801 - 

In [93]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights()


encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)


# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encode the input sequence to get the internal state vectors.
    states_value = encoder_model.predict(input_seq)

    # generate empty target sequence of length 1 with only the start character
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # output sequence loop
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

        # sample a token and add the corresponding character to the 
        # decoded sequence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # check for the exit condition: either hitting max length
        # or predicting the 'stop' character
        if (sampled_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_length):
                stop_condition = True

        # update the target sequence (length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # update states
        states_value = [h, c]

    return decoded_sentence

In [94]:
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Fire!
Decoded sentence: Feuer!

-
Input sentence: Help!
Decoded sentence: Hilfe!

-
Input sentence: Help!
Decoded sentence: Hilfe!

-
Input sentence: Stop!
Decoded sentence: Stopp!

-
Input sentence: Wait!
Decoded sentence: Warte!

-
Input sentence: Go on.
Decoded sentence: Mach weiter.

-
Input sentence: Hello!
Decoded sentence: Hallo!



In [95]:
input_sentence = "How are you?"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
    test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

How are you?
Wie geht es dir?



In [96]:
val_input_texts = []
val_target_texts = []
line_ix = 12000
for line in lines[line_ix:line_ix+10]:
    input_text, target_text = line.split('\t')
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)

val_encoder_input_data = np.zeros(
  (len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(val_input_texts):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.

In [97]:
for seq_index in range(10):
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', val_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence[:-1])
    print('Ground Truth sentence:', val_target_texts[seq_index])

-
Input sentence: I bought the car.
Decoded sentence: Ich habe es gekauft.
Ground Truth sentence: Ich habe das Auto gekauft.
-
Input sentence: I bowed politely.
Decoded sentence: Ich habe es gekauft.
Ground Truth sentence: Ich verbeugte mich hÃ¶flich.
-
Input sentence: I broke that one.
Decoded sentence: Ich habe dan dufÃ¼r gesorgen.
Ground Truth sentence: Ich habe das zerbrochen.
-
Input sentence: I came to say hi.
Decoded sentence: Ich bin ein Buch.
Ground Truth sentence: Ich bin gekommen, um Hallo zu sagen.
-
Input sentence: I came yesterday.
Decoded sentence: Ich bin statt Tom gekommen.
Ground Truth sentence: Ich bin gestern gekommen.
-
Input sentence: I can bake bread.
Decoded sentence: Ich kann es mal virs ausgugen.
Ground Truth sentence: Ich kann Brot backen.
-
Input sentence: I can be trusted.
Decoded sentence: Ich kann es beweisen.
Ground Truth sentence: Man kann mir vertrauen.
-
Input sentence: I can carry that.
Decoded sentence: Ich kann es mal versuchen.
Ground Truth senten